In [1]:
!pip install transformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 47.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 50.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 56.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.3/225.3 kB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.1/519.1 kB 51.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 10.4 MB/s eta 0:00:00


In this notebook, I leverage the causal nature of 'gpt-2' model to behave like a chatbot to which we can chat.

In [13]:
## Import necessary libraries
import torch
from torch.utils.data import Dataset, DataLoader
import json
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from torch.optim import AdamW
from transformers import DataCollatorForSeq2Seq

## Tokenize the dataset
class TokenizedDataset(Dataset):
    def __init__(self, path, tokenizer):
        self.data = json.load(open(path, "r"))
        self.tokenizer = tokenizer

        self.conversations = []
        for intent in self.data['intents']:
            patterns = intent['patterns']
            responses = intent['responses']
            for i in range(len(patterns)):
                pattern = patterns[i]
                response = responses[i % len(responses)]  # Handle cases where there are more patterns than responses
                conversation = "<startofstring> " + pattern + " <bot>: " + response + " <endofstring>"
                self.conversations.append(conversation)

        self.x_encoded = tokenizer.batch_encode_plus(
            self.conversations,
            max_length=64,
            truncation=True,
            padding='max_length',
            return_tensors='pt'
        )
        self.input_ids = self.x_encoded['input_ids']
        self.attention_mask = self.x_encoded['attention_mask']

    def __len__(self):
        return len(self.conversations)

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_mask[idx]
        }

device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.add_special_tokens({"pad_token":"<pad>",
                            "bos_token":"<startofstring>",
                            "eos_token":"<endofstring>"})
tokenizer.add_tokens(['<bot>:'])

## Initialize the model
model = GPT2LMHeadModel.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))
model.to(device)

optimizer = AdamW(model.parameters(), lr = 1e-4)

dataset = TokenizedDataset(path='intents.json', tokenizer=tokenizer)

In [14]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer)
dataloader = DataLoader(dataset, batch_size=8,collate_fn=data_collator, shuffle=True)

I have used a technique called Mixed precision training to reduce the computational resource required for training the model.

## Mixed Precision Training
Mixed precision training is a technique used in deep learning to speed up training times and reduce memory usage by using a combination of lower and higher-precision floating-point numbers. This technique involves using half-precision (16-bit) floating-point numbers for the majority of the model's computations, and then switching to single-precision (32-bit) floating-point numbers for certain parts of the computation that require higher precision.

* Autocasting is a feature provided by TensorFlow that automatically casts variables to the appropriate data type for a given operation. This feature can be used to perform mixed precision training by allowing the model to use half-precision floating-point numbers for most of the computation, while automatically promoting certain variables to single-precision when necessary.

* GradScaler is a technique used to prevent underflow and overflow errors that can occur when using half-precision floating-point numbers. This technique involves scaling the gradients during backpropagation to prevent them from becoming too small or too large. By using GradScaler in combination with autocasting, it is possible to achieve AMP (Automatic Mixed Precision) training, which can significantly speed up training times and reduce memory usage while maintaining high levels of accuracy.

In [15]:
from torch.optim import AdamW
from torch.cuda.amp import autocast, GradScaler
import tqdm

scaler = GradScaler()

def infer(inp):
  model.eval()
  inp = "<startofstring> "+inp+" <bot>: "
  inp = tokenizer(inp,return_tensors='pt')
  X = inp['input_ids'].to(device)
  a = inp['attention_mask'].to(device)
  output = model.generate(X, attention_mask=a)
  output = tokenizer.decode(output[0],skip_special_tokens=True)
  return output


def train(dataset, model, optim):
  num_epochs = 100
  model.to(device)
  model.train()
  for i in tqdm.tqdm(range(num_epochs)):
    for batch in dataset:
      X = batch['input_ids'].to(device)
      a = batch['attention_mask'].to(device)
      optim.zero_grad()
      with autocast(): ## for mixed precision training
        out = model(X, attention_mask=a,labels=X)
        loss=out.loss
      scaler.scale(loss).backward()
      scaler.step(optim)
      scaler.update()
    print(infer('Hello?'))


train(dataloader,model, optimizer)

  1%|          | 1/100 [00:01<02:39,  1.61s/it]

Hello? <bot>: 
 <bot>:  the the the the


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
  2%|▏         | 2/100 [00:02<02:11,  1.35s/it]

Hello? <bot>:


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
  3%|▎         | 3/100 [00:03<01:53,  1.17s/it]

...

Hello? <bot>: You're welcome!


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
 99%|█████████▉| 99/100 [01:46<00:00,  1.01it/s]

Hello? <bot>: You're welcome!


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
100%|██████████| 100/100 [01:47<00:00,  1.07s/it]

Hello? <bot>: You're welcome!


In [16]:
print("infer from model : ")
while True:
  inp = input()
  print(infer(inp))

infer from model : 
How are you?


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


How are you? <bot>: I'm just a machine learning model, so I don't have feelings
Do you know about any books?


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Do you know about any books? <bot>: I recommend '1984' by George Orwell. It's
How is the weather?


...

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Tell me a joke <bot>: Sure, here's a joke: Why don't scientists trust atoms?
Why?


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Why? <bot>: I'm sorry, I am an AI language model and cannot provide real-time


KeyboardInterrupt: ignored

The performance of this GPT-chatbot can obviously improved by training on more data and training for longer period of time.